In [1]:
import numpy as np

# Example of segmenting before applying the Luby transform

Correct input data is assumed

In [2]:
import numpy as np

og_data = '00000001001101111111'
segment_length = 4
nr_segments = len(og_data)//segment_length
segments= np.empty((nr_segments,1))
segments[:]=np.nan

for x in range(len(og_data)//4):
    segment = int(og_data[4*x:4*x+4],2)
    print('{0:0{1}b}'.format(segment,segment_length))

0000
0001
0011
0111
1111


# Decoding based on the Erlich paper

**This structure is suboptimal. It can only start solving when a seed with one segment is introduced. Additionally, droplets with only one nonsolved segment lead to furter solves**
- 1. Check if incoming data contains a segment which is already solved and XOR that one away
- 2. Check if the incoming data decodes for only one segment, add data and solved segment to the solved segments
- 3. repeat step 1 and 2 until the solved segments do not stop increasing
- 4. move on to new incoming data if not all segments are stored

Input data: list of tuples ( [indices of the contained segments] , "XOR of segments" )

Output data: dictionary of solved segments, index segment as key and binary segment as value

In [3]:
input_data = [([0],"0000"),
              ([1,2],"0010"),
              ([0,1],"0001"),
              ([0,2,4],"1100"),
              ([1,3,4],"1001"),
              ([0,1,2,3,4],"1010")]

nr_segments = 5
output_data = {}

def Decoding_step1and2(input_data, output_data):
    for droplet in input_data:
        segment_indices = droplet[0]
        XOR = droplet[1]
        remaining_segments = []
        for i in range(len(segment_indices)):
            if segment_indices[i] in output_data:
                dif = int(XOR,2) ^ int(output_data[segment_indices[i]],2)
                XOR = '{0:0{1}b}'.format(dif,len(droplet[1]))
            else:
                remaining_segments.append(segment_indices[i])    

        if len(remaining_segments)==1:
            output_data[remaining_segments[0]] = XOR
            newsolves=1
            while newsolves>0: 
                startsolves = len(output_data)
                Decoding_step1and2(input_data, output_data)
                endsolves = len(output_data)
                newsolves = endsolves-startsolves
    return output_data

                
output_data = Decoding_step1and2(input_data, output_data)

##add in a check to see if all segments have been solved.
#are all values from 0 to last segment keys in the dictionary
if len(output_data)==nr_segments:
    solution = ''.join([output_data[x] for x in range(nr_segments)])
else:
    solution = 'solution could not be reached, use another more robust method'
solution

'00000001001101111111'

# Decoding based on row-reduced-echelon-form using XOR

source: https://gist.github.com/sgsfak/77a1c08ac8a9b0af77393b24e44c9547
original code has been augmented to scecificaly fit the decoding of Luby transformations

each row is a droplet, the first K columns correspond to the K segments and the K+1 column is the data payload/the XOR of sequences in this droplet. For K rows(droplets) the system could possibly be solved as long as there is a one in every column (every segment is part of a droplet which has been recieved) and no duplicate rows are present.

In [7]:
import numpy as np

def rref(B, tol=1e-8):
    A = B.copy()
    rows, cols = A.shape
    r = 0
    pivots_pos = []
    for c in range(cols-1): # *pivot should not be in the last column, this is the data column*
        ## Find the pivot row:
        pivot = np.argmax(A[r:rows,c]) + r
        m = A[pivot, c]
        if m <= tol:
            ## Skip column c, making sure the approximately zero terms are
            ## actually zero.
            A[r:rows, c] = np.zeros(rows-r)
        else:
            ## keep track of bound variables
            pivots_pos.append(r)

        if pivot != r:
            ## Swap current row and pivot row
            A[[pivot, r], c:cols] = A[[r, pivot], c:cols]

        ## Eliminate the current column
        v = A[r, c:cols]
        ## Above (before row r):
        if r > 0:
            ridx_above = np.arange(r)
            for i in ridx_above:
                if A[i, c]==1:
                    A[i,c:cols] = A[i,c:cols] ^ v
        ## Below (after row r):
        if r < rows-1:
            ridx_below = np.arange(r+1,rows)
            for i in ridx_below:
                if A[i, c]==1:
                    A[i,c:cols] = A[i,c:cols] ^ v
        r += 1
        ## Check if done
        if r == rows:
            if len(pivots_pos)==r:
                break;
            else:
                print("Not enough data available to solve all segments")
                break;
    return(A, pivots_pos)


matrix = np.array([[1,0,0,0,0,0],
                   [1,0,1,0,1,12],
                   [0,1,0,1,1,9],
                   [1,1,1,1,1,10],
                   [1,1,0,0,0,1]])
result = rref(matrix)

if len(result[1])==nr_segments:
    solution = ''.join(['{0:0{1}b}'.format(x,4) for x in result[0][result[1],-1]])
else:
    solution = 'solution could not be reached, more droplets are needed'
result,solution

((array([[ 1,  0,  0,  0,  0,  0],
         [ 0,  1,  0,  0,  0,  1],
         [ 0,  0,  1,  0,  0,  3],
         [ 0,  0,  0,  1,  0,  7],
         [ 0,  0,  0,  0,  1, 15]]), [0, 1, 2, 3, 4]), '00000001001101111111')

# Initializing all possible seeds
This is not what we need to do exactly but it is just an exercise to get input for the decoding
- n: number of segments

All possible segment combinations $={2}^{n}-1$

Possible seeds for a specific index length $={2}^{indexlength}$ 

In [5]:
## set parameters and segment
og_data = '00000001001101111111'
segment_length = 4
nr_segments = len(og_data)//segment_length
segments= np.empty([nr_segments],dtype=int)

for x in range(nr_segments):
    segments[x]=(int(og_data[segment_length*x:segment_length*x+segment_length],2))

# all possible segment combinations
nr_combinations = 2**nr_segments-1
index_length = 0
while 2**index_length < nr_combinations:
    index_length = index_length+1

## Create droplets    
droplets = []
for x in range(1,nr_combinations+1):
    seed='{0:0{1}b}'.format(x,index_length)
    segment_indices = [i == '1' for i in seed]
    a = segments[segment_indices]
    payload = a[0]
    for i in range(len(a)-1):
        payload = payload ^ a[i+1]
    droplets.append(''.join([seed,'{0:0{1}b}'.format(payload,segment_length)]))

In [8]:
## Reading droplets into matrix
matrix = np.zeros([len(droplets),nr_segments+1], dtype=int)
for i in range(len(droplets)):
    seed = droplets[i][0:nr_segments]
    payload = droplets[i][nr_segments:]
    segment_indices= [x == '1' for x in seed]
#     print(segment_indices)
    matrix[i,:nr_segments] = 1*segment_indices
    matrix[i,nr_segments] = int(payload,2)

## Solve matrix to RREF
result = rref(matrix[[1,5,6,9,30],:]) #Only taking a few droplets to solve

## Check solution
if len(result[1])==nr_segments:
    solution = ''.join(['{0:0{1}b}'.format(x,4) for x in result[0][result[1],-1]])
    if solution == og_data:
        print("succesfull recovery of sequence:",solution)
    else:
        print("enough droplets collected but solution is incorrect")
else:
    solution = 'solution could not be reached, more droplets are needed'
    print(solution)


succesfull recovery of sequence: 00000001001101111111
